In [7]:
# 필요한 라이브러리 로드
import os
from data_loading import load_images_and_annotations
from data_processing import convert_annotations
from prepare_dataset import prepare_dataset, create_yaml

In [8]:
# 데이터 로드 및 전처리
train_image_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Training\[Training 원천]underwater photo'
train_annotation_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Training\[Training 라벨링]underwater photo'
val_image_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Validation\[Val 원천]underwater photo'
val_annotation_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Validation\[Val 라벨링]underwater photo'

print("Loading and processing training data...")
train_images, train_annotations = load_images_and_annotations(train_image_dir, train_annotation_dir)
print("Loading and processing validation data...")
val_images, val_annotations = load_images_and_annotations(val_image_dir, val_annotation_dir)

all_annotations = train_annotations + val_annotations
print("Converting annotations...")
converted_annotations = convert_annotations(all_annotations)

# 데이터셋 준비
output_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset'
print("Preparing dataset...")
prepare_dataset(train_images + val_images, converted_annotations, output_dir)

# 클래스 레이블 정의
classes = ['타이어', '통발류', '어망류', '나무', '로프']
create_yaml(output_dir, classes)


Loading and processing training data...
Loading images and annotations from C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Training\[Training 원천]underwater photo and C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Training\[Training 라벨링]underwater photo
Loaded 9022 images and 9022 annotations.
Loading and processing validation data...
Loading images and annotations from C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Validation\[Val 원천]underwater photo and C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Validation\[Val 라벨링]underwater photo
Loaded 3008 images and 3008 annotations.
Converting annotations...
Converting annotations to new class labels...
Converted 12030 annotations.
Preparing dataset...
Dataset prepared with 9624 training and 2406 validation images.
YAML file created at C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset\data.yaml


In [9]:
# YOLOv8 설치
# !pip install ultralytics


In [10]:
train_image_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Training\[Training 원천]underwater photo'
train_annotation_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Training\[Training 라벨링]underwater photo'
val_image_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Validation\[Val 원천]underwater photo'
val_annotation_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\Validation\[Val 라벨링]underwater photo'

output_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset'

In [29]:
import os
from ultralytics import YOLO

# output_dir 변수 정의 확인
output_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset'
print(f"Output directory is set to: {output_dir}")

# 모델 가중치를 저장할 디렉토리 생성
weights_dir = os.path.join(output_dir, 'weights')
os.makedirs(weights_dir, exist_ok=True)
print(f"Model weights will be saved to: {weights_dir}")

# YOLOv8 학습 실행
print("Starting YOLOv8 training...")
model = YOLO('yolov8n.pt')

# Early Stopping을 적용하고 학습 모니터링
results = model.train(
    data=os.path.join(output_dir, 'data.yaml'),
    epochs=50,
    imgsz=512,
    batch=16,
    name="yolo_underwater_trash",
    patience=5,  # Early stopping을 위해 5 epochs 동안 성능 향상이 없으면 중지
    device='cpu'  # CPU로 학습 강제
)

print("Training completed. Results saved in 'runs/train/yolo_underwater_trash'.")

# 매 에포크마다 가중치 저장
for epoch in range(50):
    weight_path = os.path.join(weights_dir, f'epoch_{epoch + 1}.pt')
    model.save(weight_path)
    print(f"Weights saved to {weight_path} after epoch {epoch + 1}")


Output directory is set to: C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset
Model weights will be saved to: C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset\weights
Starting YOLOv8 training...
New https://pypi.org/project/ultralytics/8.2.28 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.18  Python-3.11.7 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\dhshs\Documents\   \YOLOv8_dataset\data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=yolo_underwater_trash8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, 

train: Scanning C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset\train\labels.cache... 0 images, 9624 backgrounds, 0 corrupt: 100%|██████████| 9624/9624 [00:00<?, ?it/s]

WARNING  No labels found in C:\Users\dhshs\Documents\   \YOLOv8_dataset\train\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.



val: Scanning C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset\val\labels.cache... 0 images, 2406 backgrounds, 0 corrupt: 100%|██████████| 2406/2406 [00:00<?, ?it/s]

WARNING  No labels found in C:\Users\dhshs\Documents\   \YOLOv8_dataset\val\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.
Plotting labels to c:\Users\dhshs\Documents\CCTV\ultralytics\runs\detect\yolo_underwater_trash8\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


TensorBoard: model graph visualization added 
Image sizes 512 train, 512 val
Using 0 dataloader workers
Logging results to c:\Users\dhshs\Documents\CCTV\ultralytics\runs\detect\yolo_underwater_trash8
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G          0      82.43          0          0        512:   1%|          | 7/602 [00:19<28:50,  2.91s/it]